# Cluster Forecasting

In [1]:
import pandas as pd
import sketch
import ast

import lightgbm as lgb 
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
clients_per_clusters_path = 'clean_data/clients_per_clusters.csv'
clients_and_clusters_path = 'clean_data/clients_and_clusters.csv'
transactions_path = 'train_data_npo/npo_trnsctns.csv'
contributors_path = 'train_data_npo/npo_cntrbtrs.csv'

clients_features = {
    "clnt_id" : "ID клиента",
    "slctn_nmbr" : "Номер выборки",
    "gndr" : "Пол клиента: м: 0, ж: 1",
    "age" : "Возраст клиента, лет",
    "brth_yr" : "Год рождения клиента",
    "pstl_code" : "Почтовый индекс",
    "city" : "Живет клиент в городе (1) или нет (0)"
}

contributors_features = {
    "npo_accnt_id" : "ID счета клиента",
    "client_id" : "ID клиента",
    "slctn_nmbr" : "Номер выборки",
    "accnt_pnsn_schm" : "Условный код пенсионной схемы счета клиента",
    "npo_accnt_status" : "Статус счета клиента: Закрыт: 0, Открыт: 1",
    "npo_accnt_status_date" : "Дата статуса счета клиента",
    "npo_blnc" : "Баланс счета клиента, руб.",
    "npo_pmnts_sum" : "Сумма взносов клиента, руб.",
    "npo_pmnts_nmbr" : "Число взносов клиента",
    "npo_frst_pmnt_date" : "Дата первого взноса клиента",
    "npo_lst_pmnt_date" : "Дата последнего взноса клиента",
    "npo_ttl_incm" : "Сумма дохода, начисленного на счет клиента, руб."
}

transactions_features = {
    "npo_accnt_id" : "ID счета клиента",
    "slctn_nmbr" : "Номер выборки",
    "npo_sum" : "Размер операции по счету клиента, руб.",
    "npo_operation_date" : "Дата взноса клиента",
    "npo_operation_group" : "Тип операции по счету клиента: Поступление взносов: 0, Начисление дохода: 1",
}

In [11]:
contributors = pd.read_csv('train_data_npo/npo_cntrbtrs.csv')
contributors.head(3)

,npo_accnt_id,clnt_id,accnt_pnsn_schm,slctn_nmbr,npo_accnt_status,npo_accnt_status_date,npo_blnc,npo_pmnts_sum,npo_pmnts_nmbr,npo_frst_pmnt_date,npo_lst_pmnt_date,npo_ttl_incm
0,0x90B7458B8CBFF24980DEC312BA4A1AF5,0x85390230E8955E4FA736E62B0F0E3844,1.0,0,1,2001-05-14,10158.96,2276.42,1.0,2005-08-31,2005-08-31,5638.83
1,0xC64D3161D31A8441A65224792D370CB3,0xC2B51FD4FE57F7479210FD7258DF5B0B,3.0,2,0,2018-10-30,NaN,230084.40,55.0,2013-03-07,2017-09-10,39875.30
2,0xC92F1AA5587E2348BEF17432FBD6C2E6,0x8EC850934FF06A4AA0A856CF43B8D666,4.0,1,0,2014-01-20,NaN,7921.95,8.0,2012-11-26,2013-06-30,207.50


In [12]:
clients_per_clusters = pd.read_csv(clients_per_clusters_path, index_col=0)
clients_per_clusters.head(3)

,cluster,clnt_id
0,0,"['0x896FDAA89D08B44698884BC6876C3455', '0x7F16..."
1,1,"['0x88C8CB57D2D6B14393894C0CBB8A9A4A', '0x1441..."
2,2,"['0xE161325D909F9848979ABFE4137216CE', '0xE459..."


In [13]:
clients_and_clusters = pd.read_csv(clients_and_clusters_path, index_col=0)
clients_and_clusters.head(3)

,clnt_id,cluster,age,city,gndr
0,0xD1930AC934CD0D4AB6141DF45637EFE4,3,74.0,0,1.0
1,0x25DCE99C94913C42A49F739DDA3AE81A,3,62.0,0,0.0
2,0xCF29021EFE24454693866565B7CAB0D8,3,69.0,0,1.0


In [14]:
transactions = pd.read_csv(transactions_path)
transactions.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date,npo_operation_group
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00,0
1,0x05C7DF8BA2611640BE946E29CF20C6D2,8394.05,0,2010-08-30 00:00:00,1
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00,0


In [15]:
client_payments = transactions.loc[transactions['npo_operation_group'] == 0, 
    ['npo_accnt_id', 'npo_sum', 'slctn_nmbr', 'npo_operation_date']]
# Rename column
#client_payments.rename(columns={'npo_accnt_id':'clnt_id'}, inplace=True)
#client_payments['clnt_id'] = client_payments['clnt_id'].astype(str)
client_payments.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00
4,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2016-11-05 00:00:00


In [16]:
client_payments = pd.merge(client_payments, contributors.loc[:,['clnt_id', 'npo_accnt_id']], on='npo_accnt_id')
client_payments.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date,clnt_id
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00,0xE1494CDD114361469D1979AB0311F504
1,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00,0xE1494CDD114361469D1979AB0311F504
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2016-11-05 00:00:00,0xE1494CDD114361469D1979AB0311F504


In [17]:
client_payments = pd.merge(client_payments, clients_and_clusters, on='clnt_id')
client_payments.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date,clnt_id,cluster,age,city,gndr
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00,0xE1494CDD114361469D1979AB0311F504,3,59.0,0,0.0
1,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00,0xE1494CDD114361469D1979AB0311F504,3,59.0,0,0.0
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2016-11-05 00:00:00,0xE1494CDD114361469D1979AB0311F504,3,59.0,0,0.0


In [18]:
# Format npo_operation_date to quarter
client_payments['quarter'] = pd.to_datetime(client_payments['npo_operation_date']).dt.to_period('Q')

In [22]:
quarterly_data = client_payments.groupby(['quarter', 'slctn_nmbr', 'cluster']).agg({'npo_sum': 'sum', 'age':'mean', 'clnt_id': 'count'})
quarterly_data.reset_index(inplace=True)
quarterly_data.tail()
quarterly_data = quarterly_data.loc[quarterly_data['quarter'] >= pd.Period('2005Q1')].reset_index(drop=True)
quarterly_data.head(10)

,quarter,slctn_nmbr,cluster,npo_sum,age,clnt_id
0,2005Q1,0,0,1926.81,53.000000,12
1,2005Q1,0,3,67464.22,53.735751,193
2,2005Q1,1,0,610404.70,64.623813,1369
3,2005Q1,1,1,16968.64,56.761905,42
4,2005Q1,1,2,6050.32,50.200000,20
5,2005Q1,1,3,10220712.79,60.659935,1535
6,2005Q1,2,0,1219.50,56.500000,6
7,2005Q1,2,1,90730.80,59.075130,386
8,2005Q1,2,2,471563.38,65.505515,1632
9,2005Q1,2,3,9347330.10,58.305402,41005


In [23]:
quarterly_data['quarter'] = quarterly_data['quarter'].dt.to_timestamp()
quarterly_data['quarter']

0      2005-01-01
1      2005-01-01
2      2005-01-01
3      2005-01-01
4      2005-01-01
          ...    
1090   2022-04-01
1091   2022-04-01
1092   2022-04-01
1093   2022-04-01
1094   2022-04-01
Name: quarter, Length: 1095, dtype: datetime64[ns]

In [24]:
df_moex = pd.read_csv('df_moex', index_col=0)
df_moex['quarter'] = pd.to_datetime(df_moex['quarter'])
df_rgb = pd.read_csv('df_rgb', index_col=0)
df_rgb['quarter'] = pd.to_datetime(df_rgb['quarter'])
df_employ = pd.read_csv('df_employ', index_col=0)
df_rgb.head(3)

/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_55300/4075141266.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_moex['quarter'] = pd.to_datetime(df_moex['quarter'])
/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_55300/4075141266.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_rgb['quarter'] = pd.to_datetime(df_rgb['quarter'])


,quarter,Цена,Откр.,Макс.,Мин.,Объём,Изм. %
0,2005-01-01,11362,"114,24","114,35","113,23",NaN,"-0,64%"
1,2005-04-01,11288,"112,90","113,16","112,80",NaN,"0,00%"
2,2005-07-01,11283,"112,85","112,99","112,54",NaN,"-0,01%"


In [25]:
data_quarters = []
for _, row in df_employ.iterrows():
        year = row['year'][:4]
        bond_price = row['value']

        # Generate data for each of the four quarters of the year
        for quarter in range(1, 5):
            data_quarters.append({'quarter': f'{year}Q{quarter}', 'employ': bond_price})

    # Create the new DataFrame with quarterly values
df_employ = pd.DataFrame(data_quarters)
df_employ['quarter'] = pd.to_datetime(df_employ['quarter'])

/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_55300/3180973414.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_employ['quarter'] = pd.to_datetime(df_employ['quarter'])


In [26]:
# Join quarterly_data with df_moex on quarter
quarterly_data = pd.merge(quarterly_data, df_moex, on='quarter')
quarterly_data.rename(columns={'VALUE': 'moex'}, inplace=True)

In [27]:
# Join quarterly_data with df_moex on quarter
quarterly_data = pd.merge(quarterly_data, df_rgb.loc[:,['quarter', 'Цена']], on='quarter')

quarterly_data.rename(columns={'Цена': 'rgb_price'}, inplace=True)

#quarterly_data.drop(columns=['Объём'], inplace=True)

In [28]:
quarterly_data = pd.merge(quarterly_data, df_employ, on='quarter')
quarterly_data.head(3)

,quarter,slctn_nmbr,cluster,npo_sum,age,clnt_id,moex,rgb_price,employ
0,2005-01-01,0,0,1926.81,53.000000,12,2.452452e+10,11362,5666.0
1,2005-01-01,0,3,67464.22,53.735751,193,2.452452e+10,11362,5666.0
2,2005-01-01,1,0,610404.70,64.623813,1369,2.452452e+10,11362,5666.0


In [29]:
quarterly_data['npo_sum'] = quarterly_data['npo_sum'].astype('float')
quarterly_data['moex'] = quarterly_data['moex'].astype('float')
quarterly_data['rgb_price'] = quarterly_data['rgb_price'].astype('float')
quarterly_data['employ'] = quarterly_data['employ'].astype('float')

In [30]:
quarterly_data['quarter'].dt.year

0       2005
1       2005
2       2005
3       2005
4       2005
        ... 
1082    2022
1083    2022
1084    2022
1085    2022
1086    2022
Name: quarter, Length: 1087, dtype: int32

In [31]:
quarterly_data['year'] = quarterly_data['quarter'].dt.year
quarterly_data['month'] = quarterly_data['quarter'].dt.month
quarterly_data['day'] = quarterly_data['quarter'].dt.day

quarterly_data = quarterly_data.drop(columns=['quarter'])

quarterly_data.head(3)

,slctn_nmbr,cluster,npo_sum,age,clnt_id,moex,rgb_price,employ,year,month,day
0,0,0,1926.81,53.000000,12,2.452452e+10,11362.0,5666.0,2005,1,1
1,0,3,67464.22,53.735751,193,2.452452e+10,11362.0,5666.0,2005,1,1
2,1,0,610404.70,64.623813,1369,2.452452e+10,11362.0,5666.0,2005,1,1


In [32]:
quarterly_data.to_csv('clean_data/quarterly_data.csv')

In [33]:
qdata_0 = quarterly_data.loc[quarterly_data.cluster == 0, quarterly_data.columns != 'cluster']
qdata_0['prev_npo_sum'] = qdata_0['npo_sum'].shift(1,fill_value=qdata_0['npo_sum'].iloc[0])
qdata_1 = quarterly_data.loc[quarterly_data.cluster == 1, quarterly_data.columns != 'cluster']
qdata_1['prev_npo_sum'] = qdata_1['npo_sum'].shift(1,fill_value=qdata_1['npo_sum'].iloc[0])
qdata_2 = quarterly_data.loc[quarterly_data.cluster == 2, quarterly_data.columns != 'cluster']
qdata_2['prev_npo_sum'] = qdata_2['npo_sum'].shift(1,fill_value=qdata_2['npo_sum'].iloc[0])
qdata_3 = quarterly_data.loc[quarterly_data.cluster == 3, quarterly_data.columns != 'cluster']
#qdata_3['prev_npo_sum'] = qdata_3['npo_sum'].shift(1,fill_value=qdata_3['npo_sum'].iloc[0])
qdata_3.head(3)

,slctn_nmbr,npo_sum,age,clnt_id,moex,rgb_price,employ,year,month,day
1,0,67464.22,53.735751,193,2.452452e+10,11362.0,5666.0,2005,1,1
5,1,10220712.79,60.659935,1535,2.452452e+10,11362.0,5666.0,2005,1,1
9,2,9347330.10,58.305402,41005,2.452452e+10,11362.0,5666.0,2005,1,1


In [34]:
def train_test_split(df):
    train_size = int(len(df) * 0.8)
    train_data, test_data = df.iloc[:train_size, :], df.iloc[train_size:, :]
    X_train = train_data.loc[:, df.columns != 'npo_sum']
    y_train = train_data.loc[:, 'npo_sum']
    X_test = test_data.loc[:, df.columns != 'npo_sum']
    y_test = test_data.loc[:, 'npo_sum']
    return X_train, y_train, X_test, y_test

## Forecasting using LightGBM

In [3]:
quarterly_data = pd.read_csv('clean_data/quarterly_data.csv', index_col=0)
quarterly_data.head(3)

,slctn_nmbr,cluster,npo_sum,age,clnt_id,moex,rgb_price,employ,year,month,day
0,0,0,1926.81,53.000000,12,2.452452e+10,11362.0,5666.0,2005,1,1
1,0,3,67464.22,53.735751,193,2.452452e+10,11362.0,5666.0,2005,1,1
2,1,0,610404.70,64.623813,1369,2.452452e+10,11362.0,5666.0,2005,1,1


In [4]:
dataframes_slctn_cluster = [None]*4
# Select all columns except cluster and slctn_number by specific value of cluster AND slctn_number
cluster_vals = range(4)
slctn_number_vals = range(4)

# Get the columns to keep
columns_to_keep = quarterly_data.columns.difference(['cluster', 'slctn_nmbr'])

for cluster_val in cluster_vals:
    dataframes_slctn_cluster[cluster_val] = [None] * 4
    for slctn_number_val in slctn_number_vals:
        filtered_df = quarterly_data[(quarterly_data['cluster'] == cluster_val) & (quarterly_data['slctn_nmbr'] == slctn_number_val)]
        res = filtered_df[columns_to_keep]
        res['prev_npo_sum'] = res['npo_sum'].shift(1,fill_value=res.iloc[0]['npo_sum'])
        dataframes_slctn_cluster[cluster_val][slctn_number_val] = res

dataframes_slctn_cluster[1][1].head(3)

/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_55439/188842636.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['prev_npo_sum'] = res['npo_sum'].shift(1,fill_value=res.iloc[0]['npo_sum'])
/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_55439/188842636.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['prev_npo_sum'] = res['npo_sum'].shift(1,fill_value=res.iloc[0]['npo_sum'])
/var/folders/h2/nrzj558d5nbgkp248n_sv6gm0000gn/T/ipykernel_55439/188842636.py:14: SettingWithCop

,age,clnt_id,day,employ,moex,month,npo_sum,rgb_price,year,prev_npo_sum
3,56.761905,42,1,5666.0,2.452452e+10,1,16968.64,11362.0,2005,16968.64
15,55.750000,40,1,5666.0,1.532838e+10,4,21955.21,11288.0,2005,16968.64
27,67.063158,950,1,5666.0,1.172042e+10,7,7624721.46,11283.0,2005,21955.21


In [5]:
from optuna_hypersearch_lgbm import run_optuna

In [16]:
n_trials = 100

In [ ]:
%%time
best_trials_slctn_clusters = [None]*4
for i, slctn_number in enumerate(slctn_number_vals):
    best_trials_slctn_clusters[i] = [None]*4
    for j, cluster_number in enumerate(cluster_vals):
        print("--------------")
        print(f"Current Selection Number is {slctn_number} and Cluster Number is {cluster_number}")
        best_trials = run_optuna(dataframes_slctn_cluster[slctn_number][cluster_number], n_trials=n_trials)
        best_trials_slctn_clusters[i][j] = best_trials

[I 2023-10-01 00:51:05,888] A new study created in memory with name: no-name-7cb453b4-ee53-4c4d-9e7a-cc6f3fcf6fd2


--------------
Current Selection Number is 0 and Cluster Number is 0


/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 10.0),
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-3, 10.0),
[I 2023-10-01 00:51:07,053] Trial 0 finished with values: [602077.9271578861, -703.1201848296158] and parameters: {'reg_alpha': 0.028414885895544882, 'reg_lambda': 0.8511546336292899, 'colsample_bytree': 0.9, 'subsample': 1.0, 'learning_rate': 0.01, 'max_depth': 100, 'num_leaves': 347, 'min_chil

--------------
Current Selection Number is 0 and Cluster Number is 1


[I 2023-10-01 00:54:47,053] Trial 0 finished with values: [991611.7323863634, -22.785451493600412] and parameters: {'reg_alpha': 0.5117353307398343, 'reg_lambda': 0.10393059779291876, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.006, 'max_depth': 100, 'num_leaves': 639, 'min_child_samples': 232, 'min_data_per_groups': 37}. 
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 10.0),
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial

...

[I 2023-10-01 01:35:02,298] Trial 0 finished with values: [9158550.990519483, -5.718348865242708] and parameters: {'reg_alpha': 0.07892436271315229, 'reg_lambda': 0.17171084984834406, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 0.008, 'max_depth': 100, 'num_leaves': 72, 'min_child_samples': 131, 'min_data_per_groups': 58}. 
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 10.0),
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.

--------------
Current Selection Number is 3 and Cluster Number is 2


[I 2023-10-01 01:39:35,126] Trial 0 finished with values: [140223534.52402598, -4.440193136735445] and parameters: {'reg_alpha': 0.34571655370779686, 'reg_lambda': 3.9546289461990343, 'colsample_bytree': 0.3, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 100, 'num_leaves': 715, 'min_child_samples': 66, 'min_data_per_groups': 86}. 
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 10.0),
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.

--------------
Current Selection Number is 3 and Cluster Number is 3


[I 2023-10-01 01:43:08,772] Trial 0 finished with values: [1377042.4992164336, -0.8011411548052882] and parameters: {'reg_alpha': 0.3608952164674549, 'reg_lambda': 1.3266648793040963, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.01, 'max_depth': 100, 'num_leaves': 805, 'min_child_samples': 135, 'min_data_per_groups': 75}. 
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 10.0),
/Users/snopoff/git_repos/NSPF-Financial-Behavior/optuna_hypersearch_lgbm.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_lambda": trial.

In [10]:
for i, slctn_number in enumerate(slctn_number_vals):
    for j, cluster_number in enumerate(cluster_vals):
        print(slctn_number, cluster_number)
        for trial in best_trials_slctn_clusters[slctn_number][cluster_number]:
            print(trial.values)

0 0
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
[602077.9271578861, -703.1201848296158]
0 1
[254535.01601043806, -2.093568166665265]
0 2
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
[56024331.6240795, -12.118369478875877]
0 3
[148437.8710876944, -0.7551437306581454]
[148437.8710876944, -0.7551437306581454]
[148437.8710876944, -0.7551437306581454]
[148437.8710876944, 

In [14]:
import pickle
with open("model/model_parameters_-1_-1.pkl", "wb") as f:
    pickle.dump(best_trials_slctn_clusters[2][2][0].params, f)

In [15]:
best_trials_slctn_clusters[2][2][0].params

{'reg_alpha': 0.0154616606049762,
 'reg_lambda': 4.230128579304806,
 'colsample_bytree': 0.5,
 'subsample': 0.5,
 'learning_rate': 0.01,
 'max_depth': 20,
 'num_leaves': 774,
 'min_child_samples': 12,
 'min_data_per_groups': 25}